分析fiber number网络邻接矩阵的组间差异

首先读取所有的FN网络

In [2]:
import os
import glob
import pandas as pd
import networkx as nx
import numpy as np
import torch
import scipy.stats

DATA_PATH = "D:/code/DTI_data/network_FN/"

def data_list(sample_path):
    sub_dirs = [x[0] for x in os.walk(sample_path)]
    sub_dirs.pop(0)

    data_list = []

    for sub_dir in sub_dirs:
        file_list = []
        dir_name = os.path.basename(sub_dir)
        file_glob = os.path.join(sample_path, dir_name, '*')
        file_list.extend(glob.glob(file_glob))

        for file_name in file_list:
            data_list.append([file_name, dir_name])

    return np.array(data_list)


class MRI_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __getitem__(self, idx):
        filepath, target = self.data_list[idx][0], int(self.data_list[idx][1])
        dataframe = pd.read_csv(filepath, sep="\s+", header=None)
        pic = dataframe.to_numpy()

        return pic, target, idx

    def __len__(self):
        return len(self.data_list)

    
filelist = data_list(DATA_PATH)
dataset = MRI_Dataset(filelist)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=False)

for data, target, idx in dataloader:
    x = data.numpy()
    y = target.numpy()
    idx = idx.numpy()
    
x = x.reshape(x.shape[0], -1)

In [22]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

seed = 1
cv = 20
kf = KFold(n_splits=cv, shuffle=True, random_state=seed)
acc_sum = 0
ttest_feature = []
for idx, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    x0 = x[train_idx][y[train_idx] == 0]
    x1 = x[train_idx][y[train_idx] == 1]
#     x0 = x[y == 0]
#     x1 = x[y == 1]
    t_value, p_value = scipy.stats.ttest_ind(x0, x1, axis=0, equal_var=False, nan_policy='omit')
    x_t = x[:, p_value <=0.03]
    ttest_feature.append(np.where(p_value <=0.03))
    print(x_t.shape, y.shape)

    ynew_train = y[train_idx]
    ynew_test = y[test_idx]

    # Norm
    scaler = MaxAbsScaler()
    scaler.fit(x_t[train_idx])
    xnew_train = scaler.transform(x_t[train_idx])
    xnew_test = scaler.transform(x_t[test_idx])
    print(x_t[train_idx][0])
    print(xnew_train[0])

    # SVC
    svc = SVC(kernel='rbf', random_state=1, gamma=0.01, C=10)
    model = svc.fit(xnew_train, ynew_train)

    predict = model.predict(xnew_test)
    correct = np.sum(predict == ynew_test)
    accuracy = correct / test_idx.size
    print("cv: {}/{}, acc.: {:.1f}\n".format(idx, cv, accuracy*100))
    acc_sum += accuracy
print("total acc.: {:.1f}\n".format(acc_sum / cv * 100))

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


(98, 56) (98,)
[  0.   0.   5.   0.   5.   0.   0.   0.  13.   0.   0.   1.  31.  13.
  31.  15.  15.   3.   5.   2.   0.  20. 110.  12.   0. 110.   0.   3.
 107.   7.  20.   3.  12.   0.   4.   0.   0.   0.   0.   3.   7.   0.
   0.   0.   2.   4.   0.   4.   1.   5.   0.   0.   4.   4.   4. 107.]
[0.         0.         0.5        0.         0.5        0.
 0.         0.         0.08552632 0.         0.         0.03571429
 0.24031008 0.08552632 0.24031008 0.16666667 0.16666667 0.125
 0.13513514 0.4        0.         0.37037037 0.56410256 0.34285714
 0.         0.56410256 0.         0.125      0.92241379 0.22580645
 0.37037037 0.10344828 0.34285714 0.         0.04878049 0.
 0.         0.         0.         0.10344828 0.22580645 0.
 0.         0.         0.4        0.17391304 0.         0.36363636
 0.03571429 0.13513514 0.         0.         0.04878049 0.17391304
 0.36363636 0.92241379]
cv: 0/20, acc.: 40.0



c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


(98, 56) (98,)
[  0.   0.   5.   1.   0.   5.   0.   0.   0.  13.   0.   1.   0.   1.
  31.  13.  31.   9.   3.   2.   0.  20. 110.  12.   0.   9. 110.   0.
   3. 107.   7.  20.   3.  12.   0.   4.   0.   0.   0.   3.   0.   7.
   0.   0.   0.   0.   0.   2.   0.   2.   1.   0.   2.   0.   4. 107.]
[0.         0.         0.5        0.04761905 0.         0.5
 0.         0.         0.         0.08552632 0.         0.04761905
 0.         0.03571429 0.24031008 0.08552632 0.24031008 0.64285714
 0.125      0.4        0.         0.37037037 0.56410256 0.34285714
 0.         0.64285714 0.56410256 0.         0.125      0.92241379
 0.22580645 0.37037037 0.10344828 0.34285714 0.         0.04878049
 0.         0.         0.         0.10344828 0.         0.22580645
 0.         0.         0.         0.         0.         0.4
 0.         0.09090909 0.03571429 0.         0.09090909 0.
 0.04878049 0.92241379]
cv: 1/20, acc.: 100.0



c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


...

(98, 58) (98,)
[ 50.   0.   0.   5.   0.   0.   5.  50.   0.   0.   0.  13.   0.   0.
  31.  13.  31.  15.   9.   0.   3.   0.  11.   2.   0. 110.  12.   0.
   9. 110.   3. 107.   7.   0.   3.  12.  32.   0.  32.   4.   0.   0.
   0.   3.   0.  15.   7.   0.   0.   0.   2.   0.   0.   0.   0.  11.
   4. 107.]
[0.57471264 0.         0.         0.5        0.         0.
 0.5        0.57471264 0.         0.         0.         0.08552632
 0.         0.         0.24031008 0.08552632 0.24031008 0.17647059
 0.64285714 0.         0.125      0.         0.35483871 0.4
 0.         0.61452514 0.34285714 0.         0.64285714 0.61452514
 0.125      0.92241379 0.22580645 0.         0.10344828 0.34285714
 0.25       0.         0.25       0.04878049 0.         0.
 0.         0.10344828 0.         0.17647059 0.22580645 0.
 0.         0.         0.4        0.         0.         0.
 0.         0.35483871 0.04878049 0.92241379]
cv: 17/20, acc.: 60.0



c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


(98, 58) (98,)
[  0.   5.   0.   0.   5.   0.   0.   0.  13.   0.   0.   1.  31.  13.
  31.   4.  15.   4.   9.   3.   2.   0.  20. 110.  12.   0.   9. 110.
   0.   0.   3. 107.   7.  20.   3.  12.  12.   4.  12.   0.   0.   3.
   0.  15.   7.   0.   0.   0.   0.   2.   1.   0.  11.   0.   4.  11.
 107.   0.]
[0.         0.5        0.         0.         0.5        0.
 0.         0.         0.08552632 0.         0.         0.03571429
 0.24031008 0.08552632 0.24031008 0.09090909 0.17647059 0.09090909
 0.64285714 0.125      0.4        0.         0.37037037 0.56410256
 0.34285714 0.         0.64285714 0.56410256 0.         0.
 0.125      0.92241379 0.22580645 0.37037037 0.10344828 0.34285714
 0.17647059 0.04878049 0.17647059 0.         0.         0.10344828
 0.         0.17647059 0.22580645 0.         0.         0.
 0.         0.4        0.03571429 0.         0.23404255 0.
 0.04878049 0.23404255 0.92241379 0.        ]
cv: 18/20, acc.: 50.0



c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


(98, 48) (98,)
[ 50.   0.   5.   0.   5.  50.   0.   0.   0.  13.   0.   0.   1.  31.
  13.  31.   3.   2.   0.  20. 110.  12.   0. 110.   0.   3.   7.  20.
   3.  12.  40.   0.   4.   0.  40.   0.   0.   3.   0.   7.   0.   0.
   0.   2.   1.   0.   0.   4.]
[0.57471264 0.         0.5        0.         0.5        0.57471264
 0.         0.         0.         0.08552632 0.         0.
 0.03571429 0.24031008 0.08552632 0.24031008 0.125      0.4
 0.         0.37037037 0.56410256 0.34285714 0.         0.56410256
 0.         0.125      0.22580645 0.37037037 0.10344828 0.34285714
 0.25477707 0.         0.04878049 0.         0.25477707 0.
 0.         0.10344828 0.         0.22580645 0.         0.
 0.         0.4        0.03571429 0.         0.         0.04878049]
cv: 19/20, acc.: 50.0

total acc.: 57.0



8个假阳，2个假阴  
全体样本t检验  
[  10,   74,  185,  187,  188,  226,  246,  279,  294,  452,  632,
        722,  813,  900,  980, 1008, 1047, 1215, 1241, 1302, 1631, 1823,
       1929, 2011, 2050, 2090, 2163, 2667, 2691, 2732, 2812, 2910, 2947,
       3105, 3286, 3369, 3506, 3531, 3547, 3565, 3599, 3672, 3794, 3825,
       3828, 3915, 3919, 4063, 4084, 4092, 4093, 4116, 4142, 4176, 4228,
       4297, 4362, 4386, 4453, 4552, 4673, 4730, 4946, 4989, 5141, 5159,
       5308, 5379, 5917, 5942, 5985, 5988, 6021, 6062, 6077, 6095, 6115,
       6269, 6281, 6322, 6403, 6459, 6520, 6657, 6660, 6837, 6924, 7191,
       7210, 7319, 7356, 7369, 7466, 7521, 7636, 7717, 7778, 7794, 7822,
       7903, 7905, 7966, 7978, 8049]

In [92]:
tt_count = np.zeros(8100)
for feature_idx in ttest_feature:
    for idx in feature_idx:
        tt_count[idx] += 1
np.where(tt_count>5)

(array([  10,   74,  185,  187,  452,  632,  900, 1215, 1241, 1302, 2011,
        2050, 2667, 2691, 2732, 2812, 2910, 2947, 3105, 3286, 3672, 3794,
        3825, 3828, 3915, 3919, 4063, 4084, 4092, 4093, 4116, 4176, 4228,
        4297, 4362, 4386, 4453, 4552, 4673, 4730, 4946, 5159, 5379, 5985,
        5988, 6021, 6062, 6077, 6269, 6281, 6322, 6403, 6459, 6520, 6657,
        6660, 7319, 7356, 7521, 7794, 7903, 7966], dtype=int64),)